In [1]:
from scipy.linalg import qr, pinv, solve, norm
from numpy.random import randn
from numpy.linalg import lstsq
import numpy as np

In [2]:
# generate random data matrix
n,d = 6,4
X = randn(n,d)

# optional: give it linearly dependent columns
# X[:,3] = X[:,2]

# Understanding the pseudoinverse

In [4]:
# form pseudoinverse
Xd = pinv(X)

In [17]:
# X†X ≈ I_d
Xd @ X

array([[ 1.00000000e+00, -4.44089210e-16, -6.85215773e-17,
         2.22044605e-16],
       [ 2.08166817e-17,  1.00000000e+00, -1.11022302e-16,
        -5.41233725e-16],
       [ 0.00000000e+00, -6.66133815e-16,  1.00000000e+00,
        -5.55111512e-16],
       [ 5.55111512e-17, -1.80411242e-16, -8.32667268e-17,
         1.00000000e+00]])

In [18]:
# Returns True if two arrays are element-wise equal within a tolerance.
np.allclose(Xd @ X, np.identity(4))

True

In [19]:
# XX† !≈ I_n
X @ Xd

array([[ 0.67998416, -0.06126728,  0.02758355, -0.04062915,  0.11925649,
        -0.44409271],
       [-0.06126728,  0.97228026, -0.08862448,  0.0729915 ,  0.00945393,
        -0.09963052],
       [ 0.02758355, -0.08862448,  0.44614148,  0.47784237, -0.08884353,
        -0.04751423],
       [-0.04062915,  0.0729915 ,  0.47784237,  0.58685162,  0.08271559,
         0.01741001],
       [ 0.11925649,  0.00945393, -0.08884353,  0.08271559,  0.94436579,
         0.15327268],
       [-0.44409271, -0.09963052, -0.04751423,  0.01741001,  0.15327268,
         0.37037669]])

In [20]:
np.allclose(X @ Xd, np.identity(6))

False

In [21]:
Q,R = qr(X)
Q,R = qr(X, mode='economic')

In [10]:
np.allclose(X, Q @ R)

True

In [25]:
Q

array([[-0.63593743,  0.39918996, -0.17512136,  0.29248525],
       [ 0.15011847, -0.57769033, -0.49545855,  0.60871949],
       [-0.17308819, -0.03794072, -0.42442154, -0.48436434],
       [-0.22859713, -0.30900493, -0.45182898, -0.48472826],
       [-0.64452188, -0.5490338 ,  0.47528565,  0.04028357],
       [ 0.27463321, -0.32948384,  0.33715546, -0.26966626]])

In [28]:
R

array([[-2.31838389,  0.79354111,  0.15815649, -0.93417737],
       [ 0.        ,  2.7546115 , -0.34380807, -0.70346924],
       [ 0.        ,  0.        ,  1.00978209,  1.07517907],
       [ 0.        ,  0.        ,  0.        ,  3.12577376]])

In [29]:
print(np.allclose(Q.T @ Q, np.identity(Q.shape[1])))
Q.T @ Q

True


array([[ 1.00000000e+00, -1.14800337e-17, -1.86049288e-17,
        -7.27564627e-17],
       [-1.14800337e-17,  1.00000000e+00, -1.46577551e-16,
         5.73168480e-17],
       [-1.86049288e-17, -1.46577551e-16,  1.00000000e+00,
        -1.08279204e-17],
       [-7.27564627e-17,  5.73168480e-17, -1.08279204e-17,
         1.00000000e+00]])

In [35]:
# form data from noisy linear model
wtrue = randn(d)
y = X.dot(wtrue) + .01*randn(n)

In [36]:
# solve least squares problem to estimate w
Q,R = qr(X, mode='economic')
w = solve(R, Q.T @ y) # solve(a,b): a @ x == b for unknown x (R @ w=Q.T @y)

In [37]:
# how good is our estimate?
norm(w - wtrue)

0.013243971713760416

In [38]:
# compute mean square error
def mse(y,z):
    return sum((y-z)**2)/len(y)
    
mse(y,X.dot(w))

2.582302736553733e-06

In [41]:
# we can use the numpy.lstsq call instead
w_lstsq = np.linalg.lstsq(X, y, rcond=None)[0]
norm(w_lstsq - w)

8.804469334467935e-16

# Compute QR by hand

In [18]:
n,d = X.shape 
X0 = X.copy()
R = np.zeros((n,d))
Q = np.zeros((n,n))

# first column of Q points in direction of first column of X
r = norm(X[:,0])
Q[:,0] = X[:,0]/r
Q

array([[-0.47534812,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.02906357,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.00071364,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.45358044,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.26950649,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.70344154,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])

In [19]:
# ensure Q*R matches X on first column
R[0,0] = r

In [20]:
# verify Q*R matches X in first column
(Q@R - X)[:,0]

array([0., 0., 0., 0., 0., 0.])

In [21]:
# now delete that part from X; we've covered it already
X[:,0] -= Q[:,0]*R[0,0]

In [22]:
# verify Q*R + X = X0
np.isclose(Q@R + X, X0)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [23]:
# eliminate component of other columns in direction of first column of Q 
for j in range(1,d):
    R[0,j] = Q[:,0].dot(X[:,j])
    X[:,j] -= Q[:,0]*R[0,j]
R

array([[ 3.74544392, -0.69063861,  0.96961492,  0.82682362],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [24]:
# verify Q*R + X = X0
np.isclose(Q@R + X, X0)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [25]:
# now for all the columns!
X = X0.copy()
Q *= 0
R *= 0

# compute the QR decomposition
for i in range(d):
    r = norm(X[:,i])
    Q[:,i] = X[:,i]/r
    for j in range(i,d):
        R[i,j] = Q[:,i].dot(X[:,j])
        X[:,j] -= Q[:,i]*R[i,j]
    print("iteration",i,": QR + X = X0?", np.isclose(Q@R + X, X0).all())

iteration 0 : QR + X = X0? True
iteration 1 : QR + X = X0? True
iteration 2 : QR + X = X0? True
iteration 3 : QR + X = X0? True


In [26]:
"""Our very own QR function to compute the economy QR"""
def ourQR(X0):
    X = X0.copy()
    n,d = X.shape
    R = np.zeros((n,d))
    Q = np.zeros((n,n))

    # compute the QR decomposition
    for i in range(d):
        r = norm(X[:,i])
        Q[:,i] = X[:,i]/r
        for j in range(i,d):
            R[i,j] = Q[:,i].dot(X[:,j])
            X[:,j] -= Q[:,i]*R[i,j]
    return Q,R

In [31]:
# solve least squares problem to estimate w
Q,R = ourQR(X0)
w_byhand = solve(R[:d,:d], (Q.T @ y)[:d])

In [32]:
norm(w_byhand - w)

1.3822165187958571e-15